In [1]:
import sys
import os
import requests
import datetime
import json
import pandas
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics

In [9]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_rknop_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [10]:
rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [11]:
def run_query(query, subdict=None):
    result = rqs.post( f'{url}/db/runsqlquery/',
        json={ 'query': query, 'subdict': subdict } )
    if result.status_code != 200:
        sys.stderr.write( f"ERROR: got status code {result.status_code} ({result.reason})\n" )
    else:
        data = json.loads( result.text )
        if ( 'status' not in data ) or ( data['status'] != 'ok' ):
            sys.stderr.write( "Got unexpected response\n" )
            print(data['error'])
        else:
            return data['rows']

In [12]:
rows = run_query( 'SELECT "classifierId","brokerName","brokerVersion",'
                  '"classifierName","classifierParams" '
                  'FROM elasticc_brokerclassifier' )
classifier_info = {}
for row in rows:
    classifier_info[row["classifierId"]] = row                                 

In [13]:
rows = run_query( 'SELECT DISTINCT ON ("classId") "classId",description '
                  'FROM elasticc_gentypeofclassid '
                  'ORDER BY "classId"' )
classname = {}
for row in rows:
    classname[row["classId"]] = row["description"]

In [19]:
# consider descIngestTimestamp, alertSentTimestamp
#
# alertSentTimestamp -- this gets updated in the database when the original
#    alert is sent out.  It's NOT present in the dev database, but should
#    be in the real one
# descIngestTimestamp -- just when we happend to suck in the broker message
#
# alertId -- opaque fingerprint, don't sort on it, but it does correspond to
#   the original alert.  Different alertId = different source, i.e.
#   measurement of the transient.

sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  "classifierId","diaObjectId","classId","alertId","trueClassId","probability","descIngestTimestamp" '
          'FROM elasticc_view_sourceclassifications '
          'ORDER BY "diaObjectId","descIngestTimestamp","classifierId" '
          'OFFSET 100 LIMIT 100' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data
# classifiers = data.classifierId.unique()

Query start at 2022-08-26T09:24:25.428788...
Query done at 2022-08-26T09:24:40.897511


,classifierId,diaObjectId,classId,alertId,trueClassId,probability,descIngestTimestamp
0,26,1000211,10,2000422026,214,0.042767,2022-08-22T20:05:06.813Z
1,27,1000211,10,2000422026,214,0.088102,2022-08-22T20:05:06.813Z
2,25,1000211,10,2000422039,214,0.000000,2022-08-22T20:05:20.691Z
3,26,1000211,10,2000422039,214,0.028300,2022-08-22T20:05:20.691Z
4,27,1000211,10,2000422039,214,0.146958,2022-08-22T20:05:20.691Z
...,...,...,...,...,...,...,...
95,25,1000211,10,2000422057,214,0.000000,2022-08-22T20:28:07.948Z
96,26,1000211,10,2000422057,214,0.011028,2022-08-22T20:28:07.948Z
97,27,1000211,10,2000422057,214,0.197030,2022-08-22T20:28:07.948Z
98,25,1000211,10,2000422066,214,0.000000,2022-08-22T20:28:35.347Z


In [21]:
classname

{0: 'Static/Other',
 1: 'Non-Recurring',
 2: 'Recurring',
 10: 'Non-Recurring/Other',
 11: 'SN-like',
 12: 'Fast',
 13: 'Long',
 20: 'Recurring/Other',
 21: 'Periodic',
 22: 'Non-Periodic',
 110: 'SN-like/Other',
 111: 'Ia',
 112: 'Ib/c',
 113: 'II',
 114: 'Iax',
 115: '91bg',
 120: 'Fast/Other',
 121: 'KN',
 122: 'M-dwarf Flare',
 123: 'Dwarf Nova',
 124: 'uLens',
 130: 'Long/Other',
 131: 'SLSN',
 132: 'TDE',
 133: 'ILOT',
 134: 'CART',
 135: 'PISN',
 210: 'Periodic/Other',
 211: 'Cepheid',
 212: 'RR Lyrae',
 213: 'Delta Scuti',
 214: 'EB',
 215: 'LPV/Mira',
 220: 'Non-Periodic/Other',
 221: 'AGN'}